In [34]:
import pandas as pd
import torch

Data = pd.read_csv("food_wastage_classification_dataset_total.csv")
Data = Data.drop(["Category"], axis=1)
Data

,Sales,Stock Remaining (%),Avg_consumption Rate (%),Price,AVC,Label
0,93,0.07,0.75,13.55,5.15,0
1,100,0.00,1.00,8.85,3.92,0
2,81,0.19,0.77,17.73,7.54,0
3,100,0.00,0.99,45.11,24.98,0
4,99,0.01,0.88,19.97,6.71,0
...,...,...,...,...,...,...
99995,4,0.96,0.00,14.16,7.33,3
99996,4,0.96,0.01,15.76,8.84,3
99997,3,0.97,0.00,10.29,5.30,3
99998,3,0.97,0.20,17.91,7.25,3


In [35]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 64

Data = Data.drop(["AVC", "Price"], axis=1) # Greta will only have 3 input features

class FoodDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):        
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [36]:
X = Data.drop(["Label"], axis=1)
X = torch.tensor(X.values, dtype=torch.float32)
X

tensor([[9.3000e+01, 7.0000e-02, 7.5000e-01],
        [1.0000e+02, 0.0000e+00, 1.0000e+00],
        [8.1000e+01, 1.9000e-01, 7.7000e-01],
        ...,
        [3.0000e+00, 9.7000e-01, 0.0000e+00],
        [3.0000e+00, 9.7000e-01, 2.0000e-01],
        [4.0000e+00, 9.6000e-01, 1.0000e-01]])

In [37]:
Y = Data["Label"]
Y = torch.tensor(Y.values, dtype=torch.long)
Y

tensor([0, 0, 0,  ..., 3, 3, 3])

In [38]:
Dataset = FoodDataset(X, Y)
TrainingData, TestingData, ValidationData = random_split(Dataset, [0.7, 0.2, 0.1])

TrainLoader = DataLoader(TrainingData, batch_size=BATCH_SIZE, shuffle=True)
TestLoader = DataLoader(TestingData, batch_size=BATCH_SIZE, shuffle=False)
ValidationLoader = DataLoader(ValidationData, batch_size=BATCH_SIZE, shuffle=False)


In [39]:
import torch
import torch.nn as nn

class Greta(nn.Module):
    def __init__(self):
        super().__init__()

        self.FoodWastageConsultant = nn.Sequential(
            nn.Linear(3, 16),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4)
        )
        
    def forward(self, x):
        x = self.FoodWastageConsultant(x)
        return x

    def predict(self, x):
        return torch.argmax(self.FoodWastageConsultant(x))

In [40]:
from torchsummary import summary
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

Consultant      = Greta()
Optimizer       = Adam(Consultant.parameters(), lr=0.001, weight_decay=0.01)
CostFunction    = nn.CrossEntropyLoss()
Scheduler       = ExponentialLR(Optimizer, gamma=0.5) # To prevent overfitting the model we will use this to exponentially decrease the learning rate every epoch

In [41]:
from tqdm import tqdm # Another library which just shows us % done of loops

# Training Loop
NUMEPOCHS = 7

for i in range(NUMEPOCHS):
    # Training Loop
    Consultant.train()
    TrainingLoss     = 0
    TrainingAccuracy = 0

    for batch in tqdm(TrainLoader, desc="Training"):
        x, y = batch
        
        # Do the forwards pass on our model to find out the error
        y_hat = Consultant(x)

        # Calculate the cost
        Cost = CostFunction(y_hat, y)

        # Backwards pass
        Optimizer.zero_grad()
        Cost.backward()
        Optimizer.step()

        # Record Loss and Accuracy
        TrainingLoss += Cost.item()
        TrainingAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    TrainingAccuracy /= len(TrainLoader.dataset)
    TrainingLoss     /= len(TrainLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Training Loss: {(TrainingLoss / len(TrainLoader)):.4f}, Training Accuracy {TrainingAccuracy * 100}%")

    # For Validation loop it's pretty much the same
    Consultant.eval()
    ValidationLoss     = 0
    ValidationAccuracy = 0

    with torch.no_grad():
        for batch in tqdm(ValidationLoader, desc="Validating"):
            x, y = batch

            # Do the forwards pass on our model to find out the error
            y_hat = Consultant(x)

            # Calculate the cost
            ValidationCost = CostFunction(y_hat, y)

            # Record loss and Accuracy
            ValidationLoss += ValidationCost.item()
            ValidationAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    ValidationAccuracy /= len(ValidationLoader.dataset)
    ValidationLoss     /= len(ValidationLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Validation Loss: {(ValidationLoss):.4f}, Validation Accuracy { ValidationAccuracy * 100}%")

    # Adjust Learning Rate
    Scheduler.step()


Training: 100%|██████████| 1094/1094 [00:03<00:00, 291.55it/s]


Epoch 1/7, Avg. Training Loss: 0.0007, Training Accuracy 67.74714285714286%


Validating: 100%|██████████| 157/157 [00:00<00:00, 599.05it/s]


Epoch 1/7, Avg. Validation Loss: 0.2501, Validation Accuracy 94.06%


Training: 100%|██████████| 1094/1094 [00:03<00:00, 303.47it/s]


...

Epoch 6/7, Avg. Validation Loss: 0.1495, Validation Accuracy 95.64%


Training: 100%|██████████| 1094/1094 [00:03<00:00, 324.57it/s]


Epoch 7/7, Avg. Training Loss: 0.0001, Training Accuracy 95.81857142857143%


Validating: 100%|██████████| 157/157 [00:00<00:00, 748.22it/s]

Epoch 7/7, Avg. Validation Loss: 0.1489, Validation Accuracy 95.66%


In [42]:
Consultant.eval()
TestingLoss     = 0
TestingAccuracy = 0

with torch.no_grad():
    for batch in tqdm(TestLoader, desc="Testing"):
        x, y = batch

        # Do the forwards pass on our model to find out the error
        y_hat = Consultant(x)

        # Calculate the cost
        TestingCost = CostFunction(y_hat, y)

        # Record loss and Accuracy
        TestingLoss += TestingCost.item()
        TestingAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()

TestingAccuracy /= len(TestLoader.dataset)
TestingLoss     /= len(TestLoader)

print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Testing Loss: {(TestingLoss):.4f}, Testing Accuracy { TestingAccuracy * 100}%")

Testing: 100%|██████████| 313/313 [00:00<00:00, 700.20it/s]

Epoch 7/7, Avg. Testing Loss: 0.1461, Testing Accuracy 95.92500000000001%


In [43]:
torch.save(Consultant.state_dict(), "TrainedModels/Greta.pth")